In [70]:
from generate_signal import generate_X_matrix
from beamforming import beamforming_method
from generate_signal import generate_A_matrix
from generate_signal import generate_noise
from generate_signal import generate_R_hat
from music import music_method
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch


nbSources = 2 # Nombre de sources
nbSensors = 10 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = 10
T = 100
correlation_List = [0.4] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.


In [86]:
num_samples = 1000 # Nombre d'échantillons à générer

# Initialiser les tableaux pour stocker les matrices de covariance et les angles estimés
all_R_hat = np.zeros((num_samples, L, L), dtype=complex)
all_estimated_angles = []
for i in range(num_samples):
    # Générer des angles aléatoires pour chaque échantillon
    theta1 = np.random.uniform(-90, 90)
    theta2 = np.random.uniform(-90, 90)
    var1 = np.random.uniform(0,1000000000000)
    var2 = np.random.uniform(0,1000000000000)
    thetaList = [theta1, theta2]
    varList = [var1, var2]

    # Générer la matrice X
    X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, signal_noise_ratio)

    # Calcul de la matrice de covariance pour l'échantillon actuel
    R_hat = generate_R_hat(X)
    all_R_hat[i] = R_hat

    # Calcul du spectre MUSIC pour l'échantillon actuel
    estimated_angles = music_method(X, L, T, print_angles = False, draw_plot = False)
    all_estimated_angles.append(estimated_angles)

# Division des données en ensembles d'entraînement et de test
all_estimated_angles = np.array(all_estimated_angles)  # Conversion en numpy array si nécessaire
R_hat_train, R_hat_test, angles_train, angles_test = train_test_split(
    all_R_hat, all_estimated_angles, test_size=0.2, random_state=42
)


[[3.55308227e+11 1.39867757e+11]
 [1.39867757e+11 3.44120048e+11]]
[[5.56526702e+11 2.75932638e+11]
 [2.75932638e+11 8.55067021e+11]]
[[3.90848961e+11 1.66189489e+11]
 [1.66189489e+11 4.41649922e+11]]
[[8.20961077e+11 2.53465224e+11]
 [2.53465224e+11 4.89096116e+11]]
[[5.90055324e+11 2.18435731e+11]
 [2.18435731e+11 5.05399310e+11]]
[[8.62846418e+11 8.56799948e+10]
 [8.56799948e+10 5.31747406e+10]]
[[5.99177144e+11 2.87875884e+11]
 [2.87875884e+11 8.64440984e+11]]
[[3.00814075e+11 1.06127207e+11]
 [1.06127207e+11 2.34010492e+11]]
[[1.03717238e+11 7.68485385e+10]
 [7.68485385e+10 3.55877309e+11]]
[[6.88238037e+11 3.27513963e+11]
 [3.27513963e+11 9.74094268e+11]]
[[2.38763260e+11 1.88759726e+11]
 [1.88759726e+11 9.32676843e+11]]
[[7.64865981e+11 3.07516998e+11]
 [3.07516998e+11 7.72739167e+11]]
[[1.31624372e+11 7.50974476e+10]
 [7.50974476e+10 2.67789818e+11]]
[[2.61577384e+11 1.88336387e+11]
 [1.88336387e+11 8.47516756e+11]]
[[4.60896527e+11 1.68988914e+11]
 [1.68988914e+11 3.87251416e+

In [87]:
R_hat_train = torch.from_numpy(R_hat_train).to(dtype=torch.float32)
R_hat_test = torch.from_numpy(R_hat_test).to(dtype=torch.float32)
angles_train = torch.from_numpy(angles_train).to(dtype=torch.float32)
angles_test = torch.from_numpy(angles_test).to(dtype=torch.float32)
R_hat_train = R_hat_train.unsqueeze(1)

In [96]:
all_estimated_angle

array([[-66.3, -65.3, -65.1, ...,  56.1,  56.6,  58.1],
       [-50. , -49.3, -48.9, ...,  45.9,  46.4,  46.9],
       [-89.9, -89.7, -89.5, ...,  40.5,  43.1,  69.2],
       ...,
       [-89.7, -89.4, -88.6, ...,  17.2,  17.5,  17.7],
       [-66.3, -65.1, -64.7, ...,  35.6,  36.3,  37.9],
       [-89.6, -89. , -88.2, ...,  32. ,  32.4,  38.1]])

In [97]:
import torch
from torch.utils.data import Dataset, DataLoader

class DOADataset(Dataset):
    def __init__(self, covariance_matrices, doa_angles):
        self.covariance_matrices = covariance_matrices
        self.doa_angles = doa_angles

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
        cov_matrix = self.covariance_matrices[idx]
        angle = self.doa_angles[idx]

        # Conversion en tenseurs PyTorch
        cov_matrix = torch.from_numpy(np.real(cov_matrix)).float()
        angle = torch.tensor(angle).float()

        return cov_matrix, angle

# Création des instances de DOADataset
train_dataset = DOADataset(R_hat_train, angles_train)
test_dataset = DOADataset(R_hat_test, angles_test)

# Création des DataLoaders pour l'entraînement et le test
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [128]:
all_estimated_angles_tensor = torch.cat([angles_train, angles_test], dim=0)

# Calcul de l'angle maximum
max_angle = torch.max(all_estimated_angles_tensor)

# Calcul de la résolution ρ comme la plus petite différence entre les angles successifs triés
# Aplatir le tensor et trier les valeurs
sorted_angles = torch.sort(all_estimated_angles_tensor .flatten())[0]

# Calculer les différences entre les valeurs successives et trouver la plus petite différence non nulle
differences = sorted_angles[1:] - sorted_angles[:-1]
resolution = torch.min(differences[differences > 0])

print(f"Angle maximum: {max_angle.item()} degrés")
print(f"Résolution: {resolution.item()} degrés")
φmax = max_angle.item()
ρ=resolution.item()


Angle maximum: 89.9000015258789 degrés
Résolution: 0.09999847412109375 degrés


In [132]:
G = int(φmax // ρ)
G

899.0

In [160]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DoACNN(nn.Module):
    def __init__(self, input_size=10, φmax=90, ρ=1):
        super(DoACNN, self).__init__()
        G = int(φmax // ρ)
        num_classes = 2 * G + 1  # Calcul de la taille de sortie basée sur φmax et ρ
        self.input_size = input_size

        self.conv1 = nn.Conv2d(1, 256, kernel_size=3, stride=2, padding=1)
        # Ajoutez ici les couches convolutives supplémentaires si nécessaire...

        self.flatten = nn.Flatten()
        self.fc1_input_size = self._calculate_fc1_size()
        self.fc1 = nn.Linear(self.fc1_input_size, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 1024)
        self.fc4 = nn.Linear(1024, 100)

    def _calculate_fc1_size(self):
        # Calculer dynamiquement la taille d'entrée de fc1 basée sur les dimensions de sortie des couches convolutives
        output_size = self.calculate_output_size(self.input_size, 3, 2, 1)
        return 256 * output_size * output_size

    @staticmethod
    def calculate_output_size(input_size, kernel_size, stride, padding):
        return (input_size - kernel_size + 2 * padding) // stride + 1

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # Appliquez les autres couches convolutives...
        x = self.flatten(x)
        x = x.view(-1, self.fc1_input_size)  # Aplatir les données pour les couches FC
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  # Utiliser sigmoid pour la sortie de la classification multilabel
        return x

# Exemple d'utilisation
model = DoACNN(input_size=10, φmax=φmax, ρ=ρ)
print(model)


DoACNN(
  (conv1): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=6400, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=100, bias=True)
)


In [161]:
# Définition du modèle

# Fonction de perte et optimiseur
criterion = nn.MSELoss()  # Mean Squared Error Loss pour une tâche de régression
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Optimiseur Adam avec un taux d'apprentissage de 0.001

# Boucle d'entrainement
epochs = 10  # Nombre d'époques pour l'entrainement
for epoch in range(epochs):
    # Forward pass: Calculer les prédictions du modèle
    outputs = model(R_hat_train)
    loss = criterion(outputs, angles_train)
    
    # Backward pass et optimisations
    optimizer.zero_grad()  # Effacer les gradients des paramètres du modèle
    loss.backward()        # Calculer les gradients du loss par rapport aux paramètres
    optimizer.step()       # Mettre à jour les paramètres du modèle
    
    # Afficher les informations d'entrainement
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Évaluation du modèle
# Passage en mode évaluation pour désactiver les couches spécifiques comme dropout ou batchnorm lors du test
model.eval()
with torch.no_grad():  # Désactiver le calcul de gradient pour l'évaluation
    predicted_angles = model(R_hat_test)
    # Calculer une métrique d'évaluation, par exemple, MSE
    test_loss = criterion(predicted_angles, angles_test)
    print(f'Test Loss: {test_loss.item()}')

Epoch [1/10], Loss: 2932.886474609375
Epoch [2/10], Loss: 2932.886474609375
Epoch [3/10], Loss: 2932.886474609375
Epoch [4/10], Loss: 2932.886474609375
Epoch [5/10], Loss: 2932.886474609375
Epoch [6/10], Loss: 2932.886474609375
Epoch [7/10], Loss: 2932.886474609375
Epoch [8/10], Loss: 2932.886474609375
Epoch [9/10], Loss: 2932.886474609375
Epoch [10/10], Loss: 2932.886474609375


RuntimeError: Given groups=1, weight of size [256, 1, 3, 3], expected input[1, 200, 10, 10] to have 1 channels, but got 200 channels instead

In [159]:
print("Dimension de la sortie du modèle (attendue) : ", angles_train.shape[1])
print("Dimension de la sortie du modèle (réelle) : ", model.fc4.out_features)

# Test du modèle avec les données simulées
outputs = model(R_hat_train)
print("Dimension de la sortie du modèle (test) : ", outputs.shape)
print("Dimension des étiquettes (test) : ", angles_train.shape)

Dimension de la sortie du modèle (attendue) :  100
Dimension de la sortie du modèle (réelle) :  1799
Dimension de la sortie du modèle (test) :  torch.Size([800, 1799])
Dimension des étiquettes (test) :  torch.Size([800, 100])


In [ ]:
R_hat_train.shape

torch.Size([800, 1, 10, 10])

In [39]:
angles_train.shape

torch.Size([800, 100])